In [1]:
import shutil
from imutils import paths
import pandas as pd
import numpy as np
import cv2
from sklearn.metrics import f1_score

from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [2]:
filename = 'train_task.zip'
extract_dir = ''
shutil.unpack_archive(filename, extract_dir)

filename_test = 'test_task.zip'
extract_dir_test = ''
shutil.unpack_archive(filename_test, extract_dir_test)

In [3]:
def extract_histogram(image, bins=(8, 8, 8)):
    image = cv2.imread(image)
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [4]:
trainPath = 'train'
testPath = 'test'

trainImagePaths = sorted(list(paths.list_images(trainPath)))
testImagePaths = sorted(list(paths.list_images(testPath)))

In [15]:
len(trainImagePaths)

1000

In [5]:
len(testImagePaths)

100

In [44]:
def read_df(imagePaths):
  df = pd.DataFrame([extract_histogram(imagePaths[0])])
  df['type_img'] = imagePaths[0].split('/')[1]

  for file in imagePaths:
    if file == imagePaths[0]:
      continue
    df_str = pd.DataFrame([extract_histogram(file)])
    df_str['type_img'] = file.split('/')[1]
    df = df.append(df_str)
  
  return df

In [43]:
trainImagePaths[0].split('/')[1]

'cat.0.jpg'

In [35]:
extract_histogram(trainImagePaths[1]).shape

(512,)

In [36]:
len(testImagePaths)

100

In [45]:
train_df = read_df(trainImagePaths)
test_df = read_df(testImagePaths)

In [46]:
train_df['type_img'].str.split('.').str.get(1)

0      0
0      1
0     10
0    100
0    101
    ... 
0     95
0     96
0     97
0     98
0     99
Name: type_img, Length: 1000, dtype: object

In [47]:
train_df['type_img_ind'] = train_df['type_img'].str.split('.').str.get(1)
test_df['type_img_ind'] = test_df['type_img'].str.split('.').str.get(1)

train_df['type_img'] = train_df['type_img'].str.split('.').str.get(0)
test_df['type_img'] = test_df['type_img'].str.split('.').str.get(0)

In [48]:
train_df['type_img'] = train_df['type_img'].replace({'cat' : 0, 'dog' : 1})
test_df['type_img'] = test_df['type_img'].replace({'cat' : 0, 'dog' : 1})

In [49]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,type_img,type_img_ind
0,0.242645,0.073122,0.000232,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.087363,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,0,0
0,0.418818,0.126781,0.000085,0.000000,0.000000,0.0,0.0,0.0,0.005085,0.076441,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000042,0.000000,0,1
0,0.356737,0.184168,0.020616,0.001619,0.000717,0.0,0.0,0.0,0.000041,0.027440,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.007726,0,10
0,0.001266,0.019995,0.011941,0.001583,0.000158,0.0,0.0,0.0,0.000000,0.002145,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.023354,0.497077,0,100
0,0.000000,0.000105,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000366,...,0.0,0.002719,0.7748,0.34338,0.038583,0.114442,0.126833,0.008469,0,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.134753,0.010071,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.004053,0.041960,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.012636,0.010754,1,95
0,0.122335,0.165961,0.041414,0.016499,0.000102,0.0,0.0,0.0,0.000991,0.147656,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,1,96
0,0.081336,0.033835,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.006635,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.004499,0.228866,1,97
0,0.598071,0.090467,0.006297,0.000019,0.000000,0.0,0.0,0.0,0.000019,0.037595,...,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.010350,1,98


In [60]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns=['type_img']), 
                                                    train_df['type_img'], test_size=0.25, random_state=4)

In [61]:
model = LinearSVC(C=1.47, random_state=4)
model.fit(X_train.drop(columns=['type_img_ind']), y_train)

LinearSVC(C=1.47, random_state=4)

In [65]:
print(model.coef_[0][152], 
      model.coef_[0][66], 
      model.coef_[0][123]
        )

-0.03901284589178978 -1.0325092019410111 3.1052456830668075e-05


In [67]:
round(3.1052456830668075e-05 , 3)

0.0

In [66]:
3.1052456830668075e-05

3.1052456830668075e-05

In [63]:
y_pred = model.predict(X_test.drop(columns=['type_img_ind']))
# y_pred = model.predict(X_train.drop(columns=['type_img_ind']))

In [64]:
f1_score(y_test, y_pred, average='macro')

0.5616883116883117

In [59]:
print(model.predict(test_df[(test_df['type_img_ind'] == '1020') & (test_df['type_img'] == 1)].drop(columns=['type_img', 'type_img_ind'])), 
      model.predict(test_df[(test_df['type_img_ind'] == '1010') & (test_df['type_img'] == 1)].drop(columns=['type_img', 'type_img_ind'])), 
      model.predict(test_df[(test_df['type_img_ind'] == '1046') & (test_df['type_img'] == 0)].drop(columns=['type_img', 'type_img_ind'])), 
      model.predict(test_df[(test_df['type_img_ind'] == '1013') & (test_df['type_img'] == 0)].drop(columns=['type_img', 'type_img_ind'])), 
      )

[0] [0] [1] [0]


In [ ]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,type_img,type_img_ind
0,0.567117,0.194313,0.009939,0.000406,0.0,0.0,0.0,0.0,0.001014,0.261450,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.154963,0.218247,0,1000
0,0.013429,0.020131,0.021416,0.000176,0.0,0.0,0.0,0.0,0.012976,0.030814,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000025,0.000000,0,1001
0,0.305423,0.080943,0.000000,0.000000,0.0,0.0,0.0,0.0,0.003492,0.090926,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000258,0.024489,0,1002
0,0.399897,0.002392,0.000000,0.000000,0.0,0.0,0.0,0.0,0.110738,0.052461,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.010275,0.214028,0,1003
0,0.693011,0.144219,0.003652,0.000306,0.0,0.0,0.0,0.0,0.002686,0.131873,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000024,0.024763,0,1004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.042874,0.013570,0.000000,0.000000,0.0,0.0,0.0,0.0,0.072613,0.139533,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.017280,0.182288,1,1045
0,0.092043,0.000226,0.000000,0.000000,0.0,0.0,0.0,0.0,0.010420,0.009717,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1,1046
0,0.496956,0.048788,0.000411,0.000000,0.0,0.0,0.0,0.0,0.081550,0.128507,...,0.0,0.0,0.0,0.0,0.000037,0.000112,0.003026,0.016848,1,1047
0,0.140861,0.069838,0.000000,0.000000,0.0,0.0,0.0,0.0,0.004180,0.080936,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.020902,0.597341,1,1048


In [ ]:
test_df['type_img_ind'].value_counts()

1000    2
1037    2
1027    2
1028    2
1029    2
1030    2
1031    2
1032    2
1033    2
1034    2
1035    2
1036    2
1038    2
1001    2
1039    2
1040    2
1041    2
1042    2
1043    2
1044    2
1045    2
1046    2
1047    2
1048    2
1026    2
1025    2
1024    2
1023    2
1002    2
1003    2
1004    2
1005    2
1006    2
1007    2
1008    2
1009    2
1010    2
1011    2
1012    2
1013    2
1014    2
1015    2
1016    2
1017    2
1018    2
1019    2
1020    2
1021    2
1022    2
1049    2
Name: type_img_ind, dtype: int64